In [1]:
#!pip install selenium
#!pip install bs4
#!pip install PyPDF2
#!pip install requests

In [2]:
import os
import time

from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfFileReader
import requests
from selenium import webdriver
from urllib.request import urlopen


# Recetas bloque 1

In [35]:
PAGE_URL = "https://www.saboresdehoy.com/recetas"

In [36]:
def getLinks(url):
    PATIENCE_TIME = 60
    driver = webdriver.Chrome("C:/Users/Angel/Downloads/chromedriver-win64/chromedriver")
    driver.get(PAGE_URL)
    driver.maximize_window()

    loadMoreButton = driver.find_element_by_xpath("//div[@id='barracookies']/a[1]")
    time.sleep(2)
    loadMoreButton.click()

    while True:
        try:
            loadMoreButton = driver.find_element_by_id("loadMore")
            time.sleep(2)
            loadMoreButton.click()
            time.sleep(5)
        except Exception as e:
            print(e)
            break

    time.sleep(10)

    links = []

    links_elements = driver.find_elements_by_xpath("//a[contains(@href,'recetas/')]")

    for link in links_elements:
        links.append(link.get_attribute("href"))
    
    driver.quit()

    return(set(links))

In [5]:
def getRecipe(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    nombre_tag = soup.find("span", attrs={"class": "titulo"})
    nombre = nombre_tag.text.strip()
    
    listado_ingredientes = soup.find_all("span", attrs={"itemprop": "ingredients"})
    ingredientes = [ingrediente.text.strip() for ingrediente in listado_ingredientes]
    
    instrucciones_tag = soup.find("span", attrs={"class": "ingredientes", "itemprop": "recipeInstructions"})
    instrucciones = [p_tag.text.strip() for p_tag in instrucciones_tag.find_all("p")] if instrucciones_tag else []
    
    return nombre, ingredientes, instrucciones

In [6]:
url_recetas = getLinks(PAGE_URL)    

Message: element not interactable
  (Session info: chrome=124.0.6367.119)



In [13]:
os.makedirs("recetas", exist_ok=True)

for url in url_recetas:
    nombre, ingredientes, instrucciones = getRecipe(url)
    recipe_name_for_file = nombre.replace(" ", "_").replace("/", "-")
    recipe_file_path = os.path.join("recetas", "receta_{}.txt".format(recipe_name_for_file))
    with open(recipe_file_path, "w", encoding="utf-8") as file:
        file.write("Receta: {}\n".format(nombre))
        file.write("Ingredientes:\n")
        for ingredient in ingredientes:
            file.write("- {}\n".format(ingredient))
        file.write("Instrucciones:\n")
        for instruccion in instrucciones:
            file.write("{}\n".format(instruccion))

# Recetas bloque 2

In [2]:
PAGE_URL = "https://www.recetasnestle.com.pe/categorias/sazon-peruana"

In [3]:
def getLinks(url):
    PATIENCE_TIME = 60
    driver = webdriver.Chrome("C:/Users/Angel/Downloads/chromedriver-win64/chromedriver")
    driver.get(PAGE_URL)
    driver.maximize_window()

    while True:
        try:
            loadMoreButton = driver.find_element_by_id("load_more")
            time.sleep(2)
            loadMoreButton.click()
            time.sleep(5)
        except Exception as e:
            print(e)
            break

    time.sleep(10)

    links = []

    links_elements = driver.find_elements_by_xpath("//a[contains(@href,'recetas/')]")

    for link in links_elements:
        links.append(link.get_attribute("href"))
        
    driver.quit()

    return(set(links))


In [4]:
def getRecipe(url):
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")

    nombre = soup.find('h1', class_='hl-1').text.strip()

    listado_ingredientes = soup.find('div', class_='recipeDetail__ingredients').find_all('li')
    ingredientes = [ingrediente.get_text(strip=True) for ingrediente in listado_ingredientes]

    steps_container = soup.find('div', class_='recipeDetail__steps')
    step_items = steps_container.find_all('div', class_='recipeDetail__stepItem')
    instrucciones = []
    for step_item in step_items:
        # Find the step text
        step_text = step_item.find('div').get_text(strip=True)
        instrucciones.append(step_text)
    return nombre, ingredientes, instrucciones


In [5]:
url_recetas = getLinks(PAGE_URL)

Message: element click intercepted: Element is not clickable at point (951, 1899)
  (Session info: chrome=124.0.6367.119)



In [6]:
os.makedirs("recetas2", exist_ok=True)

for url in url_recetas:
    nombre, ingredientes, instrucciones = getRecipe(url)
    recipe_name_for_file = nombre.replace(" ", "_").replace("/", "-")
    recipe_file_path = os.path.join("recetas2", "receta_{}.txt".format(recipe_name_for_file))
    with open(recipe_file_path, "w", encoding="utf-8") as file:
        file.write("Receta: {}\n".format(nombre))
        file.write("Ingredientes:\n")
        for ingredient in ingredientes:
            file.write("- {}\n".format(ingredient))
        file.write("Instrucciones:\n")
        for instruccion in instrucciones:
            file.write("{}\n".format(instruccion))